### Second Draft Data Cleaning

##### Linear Model that will predict score difference

This is my the notebook for my first model entirly on my own that isn't for a class so I'm going to be doing a little exploring and it might have parts that don't totally make sense.

#### Brainstorm

To start, I need to get data which means I have to decide what data I want to train and build my model off of. To start, I want to look at overall record, home/away,

#### Importing Libraries
Pandas, numpy, and matplotlib, are what I always import at the start. Will import others later if needed.

In [839]:
import sys
!{sys.executable} -m pip install -U scikit-learn

In [840]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Importing data into pandas data frames

Import into one data frame for plays, import to another for scores. We will use plays to get stats and use scores to track each game.

In [841]:
seasons = np.arange(2021, 2025)

scores_dfs = []
plays_dfs = []

for yr in seasons:
    s = pd.read_csv(f'../data/raw/Season_Scores/{yr}_scores.csv')
    p = pd.read_csv(f'../data/raw/PlaysFolder/{yr}_plays.csv')
    # scores_dfs['Season'] = yr
    # plays_dfs['Season'] = yr
    scores_dfs.append(s)
    plays_dfs.append(p)

scores = pd.concat(scores_dfs, ignore_index=True)
plays = pd.concat(plays_dfs, ignore_index=True)


In [842]:
# Old mehtod for loading single season data, used to set up the data
# scores = pd.read_csv('NFL-scores-data/Season_Scores/2024_scores.csv')
# plays = pd.cread_csv('NFL-scores-data/PlaysFolder/2024_plays.csv')

### Cleaning Up

Preseason games are removed from the data frame as it is not an accurate representation of a regular season or playoff game. Bye weeks are also removed. Pro bowl games are removed

In [843]:
is_preseason_scores = scores['Week'].str.contains(r"preseason|hall of fame", case=False, na=False)
is_bye = scores['GameStatus'].str.contains(r"bye", case=False, na=False)
is_superbowl_scores = scores['Week'].str.contains(r"super bowl", case=False, na=False)
is_probowl_scores = scores['Week'].str.contains(r"pro bowl", case=False, na=False)
scores = scores[~is_preseason_scores].copy()
scores = scores[~is_bye].copy()
scores = scores[~is_superbowl_scores].copy()

is_preseason_plays = plays['Week'].str.contains(r"preseason|hall of fame", case=False, na=False)
is_superbowl_plays = plays['Week'].str.contains(r"super bowl", case=False, na=False)
plays = plays[~is_preseason_plays].copy()
plays = plays[~is_superbowl_plays].copy()


/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_41946/274068839.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  scores = scores[~is_bye].copy()
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_41946/274068839.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  scores = scores[~is_superbowl_scores].copy()
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_41946/274068839.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  plays = plays[~is_superbowl_plays].copy()


In [844]:
TEAM_NAME_MAP = {
    # Washington
    "Washington Football Team": "Commanders",
    "Football Team": "Commanders",
    "Washington Commanders": "Commanders",
    "Commanders": "Commanders",

    # Optional future-proofing
    "Los Angeles Rams": "Rams",
    "St. Louis Rams": "Rams",
    "Oakland Raiders": "Raiders",
    "Las Vegas Raiders": "Raiders",
    "San Diego Chargers": "Chargers",
    "Los Angeles Chargers": "Chargers",
}

def standardize_team_name(name):
    return TEAM_NAME_MAP.get(name, name)

standardize_team_name('Washington Football Team')  # Example usage

def standardize_team_names_in_df(df, columns):
    for col in columns:
         df[col] = df[col].astype(str).str.strip().replace(TEAM_NAME_MAP)
    return df

scores = standardize_team_names_in_df(scores, ['HomeTeam', 'AwayTeam'])
plays = standardize_team_names_in_df(plays, ['AwayTeam', 'HomeTeam'])

In [845]:
display(scores.head())
display(plays.head())

,Season,Week,GameStatus,Day,Date,AwayTeam,AwayRecord,AwayScore,AwayWin,HomeTeam,HomeRecord,HomeScore,HomeWin,AwaySeeding,HomeSeeding,PostSeason
49,2021,Week 1,FINAL,THU,09/09,Cowboys,0-1,29.0,0.0,Buccaneers,1-0,31.0,1.0,NaN,NaN,0
50,2021,Week 1,FINAL,SUN,09/12,Jaguars,0-1,21.0,0.0,Texans,1-0,37.0,1.0,NaN,NaN,0
51,2021,Week 1,FINAL,SUN,09/12,Chargers,1-0,20.0,1.0,Commanders,0-1,16.0,0.0,NaN,NaN,0
52,2021,Week 1,FINAL,SUN,09/12,Seahawks,1-0,28.0,1.0,Colts,0-1,16.0,0.0,NaN,NaN,0
53,2021,Week 1,FINAL,SUN,09/12,Jets,0-1,14.0,0.0,Panthers,1-0,19.0,1.0,NaN,NaN,0


,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayStart,PlayTimeFormation,PlayDescription
7830,2021,Week 1,THU,09/09,Cowboys,Buccaneers,1st Quarter,1,Tampa Bay Buccaneers,0,1,0,Kickoff from DAL 35,NaN,Kickoff,— G.Zuerlein kicks 65 yards from DAL 35 to end...
7831,2021,Week 1,THU,09/09,Cowboys,Buccaneers,1st Quarter,1,Tampa Bay Buccaneers,0,2,0,5 Yard Run,1st & 10 at TB 25,15:00 1st Shotgun,— L.Fournette up the middle to TB 30 for 5 yar...
7832,2021,Week 1,THU,09/09,Cowboys,Buccaneers,1st Quarter,1,Tampa Bay Buccaneers,0,3,0,3 Yard Run,2nd & 5 at TB 30,14:17 1st,— L.Fournette right guard to TB 33 for 3 yards...
7833,2021,Week 1,THU,09/09,Cowboys,Buccaneers,1st Quarter,1,Tampa Bay Buccaneers,0,4,0,Pass Incomplete,3rd & 2 at TB 33,13:33 1st Shotgun,— T.Brady pass incomplete deep right to C.Godwin.
7834,2021,Week 1,THU,09/09,Cowboys,Buccaneers,1st Quarter,1,Tampa Bay Buccaneers,0,5,0,Punt,4th & 2 at TB 33,13:26 1st,"— B.Pinion punts 65 yards to DAL 2, Center-Z.T..."


In [846]:
keys = ['Season', 'Week', 'HomeTeam', 'AwayTeam']
games = scores[keys].drop_duplicates().reset_index(drop=True)
games['game_id'] = games.index.astype(int)
games[games['game_id']==1073]

,Season,Week,HomeTeam,AwayTeam,game_id
1073,2024,Week 15,Eagles,Steelers,1073


In [847]:
scores = scores.merge(games, on=keys, how="left")
plays  = plays.merge(games, on=keys, how="left")
scores.groupby('Week').size()

Week
Conference Championships     8
Divisional Playoffs         16
Pro Bowl                     1
Week 1                      64
Week 10                     56
Week 11                     57
Week 12                     60
Week 13                     58
Week 14                     55
Week 15                     64
Week 16                     64
Week 17                     64
Week 18                     64
Week 2                      64
Week 3                      64
Week 4                      64
Week 5                      60
Week 6                      57
Week 7                      55
Week 8                      62
Week 9                      56
Wild Card                   24
dtype: int64

In [848]:
#  Old way to set game_id but it didn't work
#  keys = ['Season', 'Week', 'HomeTeam', 'AwayTeam']
# plays['game_id'] = plays.groupby(keys).ngroup()
# scores['game_id'] = scores.groupby(keys).ngroup()
# scores['game_id'] = scores['game_id'].astype(int)
scores[scores['game_id']==2].iloc[0]

Season               2021
Week               Week 1
GameStatus          FINAL
Day                   SUN
Date                09/12
AwayTeam         Chargers
AwayRecord            1-0
AwayScore            20.0
AwayWin               1.0
HomeTeam       Commanders
HomeRecord            0-1
HomeScore            16.0
HomeWin               0.0
AwaySeeding           NaN
HomeSeeding           NaN
PostSeason              0
game_id                 2
Name: 2, dtype: object

In [849]:
rushing_plays = plays[plays['PlayOutcome'].str.contains(r"rush|run", case=False, na=False)]
rushing_plays['RushYards'] = rushing_plays['PlayOutcome'].str.extract(r"(-?\d+)\s*Yard")
rushing_plays['RushYards'] = rushing_plays['RushYards'].astype(float)

passing_plays = plays[plays['PlayOutcome'].str.contains(r"pass", case=False, na=False)]
passing_plays['PassYards'] = passing_plays['PlayOutcome'].str.extract(r"(-?\d+)\s*Yard")
passing_plays = passing_plays[~passing_plays['PassYards'].isna()]
passing_plays['PassYards'] = passing_plays['PassYards'].astype(float)

touchdown_plays = plays[plays['PlayOutcome'].str.contains(r"touchdown", case=False, na=False)]

/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_41946/2475392443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rushing_plays['RushYards'] = rushing_plays['PlayOutcome'].str.extract(r"(-?\d+)\s*Yard")
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_41946/2475392443.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rushing_plays['RushYards'] = rushing_plays['RushYards'].astype(float)
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_41946/2475392443.py:6: SettingWithCopy

In [850]:
touchdown_plays['pass'] = touchdown_plays['PlayDescription'].str.contains(r"pass", case=False, na=False).astype(int)
touchdown_plays["td_yards"] = touchdown_plays["PlayDescription"].str.extract(r"for\s+(-?\d+)\s+yard")
touchdown_plays["td_yards"] = touchdown_plays["td_yards"].dropna().astype(int)

/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_41946/1236499397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  touchdown_plays['pass'] = touchdown_plays['PlayDescription'].str.contains(r"pass", case=False, na=False).astype(int)
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_41946/1236499397.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  touchdown_plays["td_yards"] = touchdown_plays["PlayDescription"].str.extract(r"for\s+(-?\d+)\s+yard")
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0

In [851]:
rush_td_plays = touchdown_plays[touchdown_plays['pass'] == 0]
pass_td_plays = touchdown_plays[touchdown_plays['pass'] == 1]


In [852]:
rush_td_plays[rush_td_plays['game_id']==1073]

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayStart,PlayTimeFormation,PlayDescription,game_id,pass,td_yards


In [853]:
#Merge rushing and passing stats per game per team on keys
game_stats_keys = ['game_id', 'HomeTeam', 'TeamWithPossession']
rush = (rushing_plays
        .groupby(game_stats_keys, as_index=False)["RushYards"]
        .sum())

pas = (passing_plays
       .groupby(game_stats_keys, as_index=False)["PassYards"]
       .sum())
rush_td = rush_td_plays.groupby(['game_id', 'HomeTeam', 'TeamWithPossession'], as_index=False)['td_yards'].sum()
pass_td = pass_td_plays.groupby(['game_id', 'HomeTeam', 'TeamWithPossession'], as_index=False)['td_yards'].sum()
non_td_stats = rush.merge(pas, on=game_stats_keys, how="outer")
td_stats = rush_td.merge(pass_td, on=game_stats_keys, how="outer", suffixes=('_RushTD', '_PassTD'))
td_stats = td_stats.fillna(0)
display(td_stats.head(5))
display(non_td_stats.head(5))


,game_id,HomeTeam,TeamWithPossession,td_yards_RushTD,td_yards_PassTD
0,0,Buccaneers,Dallas Cowboys,0.0,38.0
1,0,Buccaneers,Tampa Bay Buccaneers,0.0,65.0
2,1,Texans,Houston Texans,6.0,15.0
3,1,Texans,Jacksonville Jaguars,0.0,67.0
4,2,Commanders,Los Angeles Chargers,3.0,3.0


,game_id,HomeTeam,TeamWithPossession,RushYards,PassYards
0,0,Buccaneers,Dallas Cowboys,60.0,355.0
1,0,Buccaneers,Tampa Bay Buccaneers,51.0,303.0
2,1,Texans,Houston Texans,165.0,276.0
3,1,Texans,Jacksonville Jaguars,66.0,280.0
4,2,Commanders,Los Angeles Chargers,87.0,334.0


In [854]:
non_td_stats[non_td_stats['RushYards'].isna()]

,game_id,HomeTeam,TeamWithPossession,RushYards,PassYards


In [855]:
game_stats = non_td_stats.merge(td_stats, on=game_stats_keys, how="outer")
game_stats = game_stats.fillna(0)
game_stats['TeamWithPossession'] = game_stats['TeamWithPossession'].str.split().str[-1]
game_stats['RushYards'] = game_stats['RushYards'] + game_stats['td_yards_RushTD']
game_stats['PassYards'] = game_stats['PassYards'] + game_stats['td_yards_PassTD']
game_stats.drop(columns=['td_yards_RushTD', 'td_yards_PassTD'], inplace=True)
game_stats[(game_stats['HomeTeam'].str.contains('Eagles')) & (game_stats['TeamWithPossession'].str.contains('Steelers'))]

,game_id,HomeTeam,TeamWithPossession,RushYards,PassYards
801,401,Eagles,Steelers,140.0,196.0


In [856]:
print(game_stats['RushYards'].isna().sum() + game_stats['PassYards'].isna().sum())

0


In [857]:
home_stats = game_stats[game_stats['HomeTeam'] == game_stats['TeamWithPossession']].drop('TeamWithPossession', axis=1)
home_stats = home_stats.rename(columns={'RushYards': 'HomeRushYards','PassYards': 'HomePassYards'})
away_stats = game_stats[game_stats['HomeTeam'] != game_stats['TeamWithPossession']].drop('TeamWithPossession', axis=1)
away_stats = away_stats.rename(columns={'RushYards': 'AwayRushYards', 'PassYards': 'AwayPassYards'})

In [858]:
display(home_stats.head())
display(away_stats.head())

,game_id,HomeTeam,HomeRushYards,HomePassYards
1,0,Buccaneers,51.0,368.0
2,1,Texans,171.0,291.0
5,2,Commanders,127.0,135.0
6,3,Colts,113.0,251.0
8,4,Panthers,111.0,279.0


,game_id,HomeTeam,AwayRushYards,AwayPassYards
0,0,Buccaneers,60.0,393.0
3,1,Texans,66.0,347.0
4,2,Commanders,90.0,337.0
7,3,Colts,139.0,254.0
9,4,Panthers,46.0,258.0


In [859]:
#home_stats['HomePassYards'].isna().sum() + home_stats['HomeRushYards'].isna().sum() + away_stats['AwayPassYards'].isna().sum() + away_stats['AwayRushYards'].isna().sum()
display(home_stats[(home_stats['HomeTeam']=='Eagles')])
display(away_stats[(away_stats['HomeTeam']=='Eagles')])
#display(away_stats[away_stats['Season']==2024])

,game_id,HomeTeam,HomeRushYards,HomePassYards
40,20,Eagles,166.0,190.0
101,50,Eagles,103.0,387.0
160,80,Eagles,100.0,115.0
263,131,Eagles,176.0,162.0
303,151,Eagles,239.0,147.0
446,223,Eagles,221.0,296.0
455,227,Eagles,145.0,189.0
515,257,Eagles,149.0,202.0
631,316,Eagles,170.0,333.0
683,342,Eagles,219.0,263.0


,game_id,HomeTeam,AwayRushYards,AwayPassYards
41,20,Eagles,148.0,189.0
100,50,Eagles,200.0,281.0
161,80,Eagles,117.0,297.0
262,131,Eagles,88.0,370.0
302,151,Eagles,108.0,265.0
447,223,Eagles,48.0,194.0
454,227,Eagles,89.0,143.0
514,257,Eagles,171.0,305.0
630,316,Eagles,62.0,221.0
682,342,Eagles,71.0,174.0


In [860]:
scores = scores.merge(home_stats[['game_id', 'HomeRushYards', 'HomePassYards']], on=['game_id'], how='left')
scores = scores.merge(away_stats[['game_id', 'AwayRushYards', 'AwayPassYards']], on=['game_id'], how='left')

In [861]:
scores[scores['HomeRushYards'].isna()] #['game_id']

,Season,Week,GameStatus,Day,Date,AwayTeam,AwayRecord,AwayScore,AwayWin,HomeTeam,...,HomeScore,HomeWin,AwaySeeding,HomeSeeding,PostSeason,game_id,HomeRushYards,HomePassYards,AwayRushYards,AwayPassYards
284,2021,Pro Bowl,FINAL,SUN,02/06,NFC Pro Bowl,NaN,35.0,0.0,AFC Pro Bowl,...,41.0,1.0,NaN,NaN,0,284,NaN,NaN,NaN,NaN
540,2022,Week 17,CANCELLED,CANCELLED,CANCELLED,Bills,12-3,0.0,0.0,Bengals,...,0.0,0.0,NaN,NaN,0,540,NaN,NaN,NaN,NaN
1073,2024,Week 15,FINAL,SUN,12/15,Steelers,10-4,13.0,0.0,Eagles,...,27.0,1.0,NaN,NaN,0,1073,NaN,NaN,NaN,NaN


In [862]:
#csv doesn't have stats for this game so I am mannualy inputing them
home_rush = 131
home_pass = 270
away_rush = 56
away_pass = 108

gid = 1073

scores.loc[gid, "HomeRushYards"] = home_rush
scores.loc[gid, "HomePassYards"] = home_pass
scores.loc[gid, "AwayRushYards"] = away_rush
scores.loc[gid, "AwayPassYards"] = away_pass

In [863]:
new_game_data_dfs = []

for yr in seasons:
    ng = pd.read_csv(f'../data/raw/GamesFolder/{yr}_games.csv')
    new_game_data_dfs.append(ng)

new_game_data = pd.concat(new_game_data_dfs, ignore_index=True)

In [864]:
new_game_data['Winner/tie'] = new_game_data['Winner/tie'].replace('Washington Football Team', 'Washington Commanders')
new_game_data['Loser/tie'] = new_game_data['Loser/tie'].replace('Washington Football Team', 'Washington Commanders')
new_game_data

,Week,Day,Date,Time,Winner/tie,@?,Loser/tie,Date.1,PtsW,PtsL,YdsW,TOW,YdsL,TOL
0,1,Thu,2021-09-09,8:20PM,Tampa Bay Buccaneers,NaN,Dallas Cowboys,boxscore,31,29,431,4,451,1
1,1,Sun,2021-09-12,1:00PM,Philadelphia Eagles,@,Atlanta Falcons,boxscore,32,6,434,0,260,0
2,1,Sun,2021-09-12,1:00PM,Pittsburgh Steelers,@,Buffalo Bills,boxscore,23,16,252,0,371,1
3,1,Sun,2021-09-12,1:00PM,Carolina Panthers,NaN,New York Jets,boxscore,19,14,381,1,252,1
4,1,Sun,2021-09-12,1:00PM,Cincinnati Bengals,NaN,Minnesota Vikings,boxscore,27,24,366,0,403,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134,Division,Sun,2025-01-19,3:00PM,Philadelphia Eagles,NaN,Los Angeles Rams,boxscore,28,22,350,0,402,2
1135,Division,Sun,2025-01-19,6:30PM,Buffalo Bills,NaN,Baltimore Ravens,boxscore,27,25,273,0,416,3
1136,ConfChamp,Sun,2025-01-26,3:00PM,Philadelphia Eagles,NaN,Washington Commanders,boxscore,55,23,459,0,350,4
1137,ConfChamp,Sun,2025-01-26,6:30PM,Kansas City Chiefs,NaN,Buffalo Bills,boxscore,32,29,368,1,374,0


In [865]:
new_game_data[((new_game_data['Winner/tie'].str.contains('Washington')) | (new_game_data['Loser/tie'].str.contains('Washington')))]

,Week,Day,Date,Time,Winner/tie,@?,Loser/tie,Date.1,PtsW,PtsL,YdsW,TOW,YdsL,TOL
9,1,Sun,2021-09-12,1:00PM,Los Angeles Chargers,@,Washington Commanders,boxscore,20,16,424,2,259,1
16,2,Thu,2021-09-16,8:20PM,Washington Commanders,NaN,New York Giants,boxscore,30,29,407,1,391,0
34,3,Sun,2021-09-26,1:00PM,Buffalo Bills,NaN,Washington Commanders,boxscore,43,21,481,0,290,3
49,4,Sun,2021-10-03,1:00PM,Washington Commanders,@,Atlanta Falcons,boxscore,34,30,412,0,374,0
73,5,Sun,2021-10-10,1:00PM,New Orleans Saints,@,Washington Commanders,boxscore,33,22,369,2,373,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,17,Sun,2024-12-29,8:20PM,Washington Commanders,NaN,Atlanta Falcons,boxscore,30,24,412,1,337,1
1116,18,Sun,2025-01-05,1:00PM,Washington Commanders,@,Dallas Cowboys,boxscore,23,19,269,1,378,0
1130,WildCard,Sun,2025-01-12,8:00PM,Washington Commanders,@,Tampa Bay Buccaneers,boxscore,23,20,350,0,284,1
1133,Division,Sat,2025-01-18,8:00PM,Washington Commanders,@,Detroit Lions,boxscore,45,31,481,0,521,5


In [866]:
new_game_data['HomeTeam'] = np.where(
    new_game_data['@?'] == '@',
    new_game_data['Loser/tie'],
    new_game_data['Winner/tie']
)
new_game_data['AwayTeam'] = np.where(
    new_game_data['@?'].isna(), 
    new_game_data['Loser/tie'], 
    new_game_data['Winner/tie'])

new_game_data['HomeTeamTO'] = np.where(
    new_game_data['@?'] == '@',
    new_game_data['TOL'],
    new_game_data['TOW']
)

new_game_data['AwayTeamTO'] = np.where(
    new_game_data['@?'].isna(), 
    new_game_data['TOL'], 
    new_game_data['TOW']) 

new_game_data['HomeTeam'] = new_game_data['HomeTeam'].str.split().str[-1]
new_game_data['AwayTeam'] = new_game_data['AwayTeam'].str.split().str[-1]
new_game_data.iloc[58]

Week                          4
Day                         Sun
Date                 2021-10-03
Time                     4:05PM
Winner/tie    Arizona Cardinals
@?                            @
Loser/tie      Los Angeles Rams
Date.1                 boxscore
PtsW                         37
PtsL                         20
YdsW                        465
TOW                           0
YdsL                        401
TOL                           2
HomeTeam                   Rams
AwayTeam              Cardinals
HomeTeamTO                    2
AwayTeamTO                    0
Name: 58, dtype: object

In [867]:
new_game_data['Date'] = pd.to_datetime(new_game_data['Date'])
new_game_data['Season'] = new_game_data['Date'].dt.year
new_game_data.loc[new_game_data['Date'].dt.month < 9, 'Season'] -= 1
new_game_data

,Week,Day,Date,Time,Winner/tie,@?,Loser/tie,Date.1,PtsW,PtsL,YdsW,TOW,YdsL,TOL,HomeTeam,AwayTeam,HomeTeamTO,AwayTeamTO,Season
0,1,Thu,2021-09-09,8:20PM,Tampa Bay Buccaneers,NaN,Dallas Cowboys,boxscore,31,29,431,4,451,1,Buccaneers,Cowboys,4,1,2021
1,1,Sun,2021-09-12,1:00PM,Philadelphia Eagles,@,Atlanta Falcons,boxscore,32,6,434,0,260,0,Falcons,Eagles,0,0,2021
2,1,Sun,2021-09-12,1:00PM,Pittsburgh Steelers,@,Buffalo Bills,boxscore,23,16,252,0,371,1,Bills,Steelers,1,0,2021
3,1,Sun,2021-09-12,1:00PM,Carolina Panthers,NaN,New York Jets,boxscore,19,14,381,1,252,1,Panthers,Jets,1,1,2021
4,1,Sun,2021-09-12,1:00PM,Cincinnati Bengals,NaN,Minnesota Vikings,boxscore,27,24,366,0,403,1,Bengals,Vikings,0,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134,Division,Sun,2025-01-19,3:00PM,Philadelphia Eagles,NaN,Los Angeles Rams,boxscore,28,22,350,0,402,2,Eagles,Rams,0,2,2024
1135,Division,Sun,2025-01-19,6:30PM,Buffalo Bills,NaN,Baltimore Ravens,boxscore,27,25,273,0,416,3,Bills,Ravens,0,3,2024
1136,ConfChamp,Sun,2025-01-26,3:00PM,Philadelphia Eagles,NaN,Washington Commanders,boxscore,55,23,459,0,350,4,Eagles,Commanders,0,4,2024
1137,ConfChamp,Sun,2025-01-26,6:30PM,Kansas City Chiefs,NaN,Buffalo Bills,boxscore,32,29,368,1,374,0,Chiefs,Bills,1,0,2024


In [868]:
new_game_data["Date"] = (
    new_game_data["Date"]
    .astype(str)
    .str.strip()
)
new_game_data['Date'] = pd.to_datetime(
    new_game_data["Date"],
    format="%Y-%m-%d",
    errors="coerce"   # invalid strings become NaT instead of crashing
)

new_game_data

,Week,Day,Date,Time,Winner/tie,@?,Loser/tie,Date.1,PtsW,PtsL,YdsW,TOW,YdsL,TOL,HomeTeam,AwayTeam,HomeTeamTO,AwayTeamTO,Season
0,1,Thu,2021-09-09,8:20PM,Tampa Bay Buccaneers,NaN,Dallas Cowboys,boxscore,31,29,431,4,451,1,Buccaneers,Cowboys,4,1,2021
1,1,Sun,2021-09-12,1:00PM,Philadelphia Eagles,@,Atlanta Falcons,boxscore,32,6,434,0,260,0,Falcons,Eagles,0,0,2021
2,1,Sun,2021-09-12,1:00PM,Pittsburgh Steelers,@,Buffalo Bills,boxscore,23,16,252,0,371,1,Bills,Steelers,1,0,2021
3,1,Sun,2021-09-12,1:00PM,Carolina Panthers,NaN,New York Jets,boxscore,19,14,381,1,252,1,Panthers,Jets,1,1,2021
4,1,Sun,2021-09-12,1:00PM,Cincinnati Bengals,NaN,Minnesota Vikings,boxscore,27,24,366,0,403,1,Bengals,Vikings,0,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134,Division,Sun,2025-01-19,3:00PM,Philadelphia Eagles,NaN,Los Angeles Rams,boxscore,28,22,350,0,402,2,Eagles,Rams,0,2,2024
1135,Division,Sun,2025-01-19,6:30PM,Buffalo Bills,NaN,Baltimore Ravens,boxscore,27,25,273,0,416,3,Bills,Ravens,0,3,2024
1136,ConfChamp,Sun,2025-01-26,3:00PM,Philadelphia Eagles,NaN,Washington Commanders,boxscore,55,23,459,0,350,4,Eagles,Commanders,0,4,2024
1137,ConfChamp,Sun,2025-01-26,6:30PM,Kansas City Chiefs,NaN,Buffalo Bills,boxscore,32,29,368,1,374,0,Chiefs,Bills,1,0,2024


In [869]:
#check to see if new_game_data has any NaN values in HomeTeamTO or AwayTeamTO, should be 0 for both
print(new_game_data[new_game_data['AwayTeamTO'].isna()].shape[0])
print(new_game_data[new_game_data['HomeTeamTO'].isna()].shape[0])

0
0


In [870]:
scores["Date"] = scores["Date"].astype(str).str.strip()

scores["Date"] = pd.to_datetime(
    scores["Season"].astype(str) + "-" + scores["Date"],
    format="%Y-%m/%d",
    errors="coerce"
)
scores["Date"] = pd.to_datetime(scores["Date"], errors="coerce")

# Jan/Feb games are in the next calendar year
mask = scores["Date"].dt.month <= 2
scores.loc[mask, "Date"] = scores.loc[mask, "Date"] + pd.DateOffset(years=1)

In [871]:
new_game_data[(new_game_data['Season']==2021)].sort_values(by='Date').head(15)

,Week,Day,Date,Time,Winner/tie,@?,Loser/tie,Date.1,PtsW,PtsL,YdsW,TOW,YdsL,TOL,HomeTeam,AwayTeam,HomeTeamTO,AwayTeamTO,Season
0,1,Thu,2021-09-09,8:20PM,Tampa Bay Buccaneers,NaN,Dallas Cowboys,boxscore,31,29,431,4,451,1,Buccaneers,Cowboys,4,1,2021
14,1,Sun,2021-09-12,8:20PM,Los Angeles Rams,NaN,Chicago Bears,boxscore,34,14,386,0,322,2,Rams,Bears,0,2,2021
13,1,Sun,2021-09-12,4:25PM,Miami Dolphins,@,New England Patriots,boxscore,17,16,259,1,393,2,Patriots,Dolphins,2,1,2021
12,1,Sun,2021-09-12,4:25PM,New Orleans Saints,NaN,Green Bay Packers,boxscore,38,3,322,0,229,3,Saints,Packers,0,3,2021
10,1,Sun,2021-09-12,4:25PM,Kansas City Chiefs,NaN,Cleveland Browns,boxscore,33,29,397,0,457,2,Chiefs,Browns,0,2,2021
9,1,Sun,2021-09-12,1:00PM,Los Angeles Chargers,@,Washington Commanders,boxscore,20,16,424,2,259,1,Commanders,Chargers,1,2,2021
8,1,Sun,2021-09-12,1:00PM,Houston Texans,NaN,Jacksonville Jaguars,boxscore,37,21,449,0,395,3,Texans,Jaguars,0,3,2021
11,1,Sun,2021-09-12,4:25PM,Denver Broncos,@,New York Giants,boxscore,27,13,420,1,314,1,Giants,Broncos,1,1,2021
6,1,Sun,2021-09-12,1:00PM,Arizona Cardinals,@,Tennessee Titans,boxscore,38,13,416,1,248,3,Titans,Cardinals,3,1,2021
5,1,Sun,2021-09-12,1:00PM,Seattle Seahawks,@,Indianapolis Colts,boxscore,28,16,381,1,336,1,Colts,Seahawks,1,1,2021


In [872]:
turnovers = new_game_data[
    ["Season", "Date", "HomeTeam", "AwayTeam", "HomeTeamTO", "AwayTeamTO"]
]

turnovers.sort_values(by=['Season', 'Date']).head(10)

,Season,Date,HomeTeam,AwayTeam,HomeTeamTO,AwayTeamTO
0,2021,2021-09-09,Buccaneers,Cowboys,4,1
1,2021,2021-09-12,Falcons,Eagles,0,0
2,2021,2021-09-12,Bills,Steelers,1,0
3,2021,2021-09-12,Panthers,Jets,1,1
4,2021,2021-09-12,Bengals,Vikings,0,1
5,2021,2021-09-12,Colts,Seahawks,1,1
6,2021,2021-09-12,Titans,Cardinals,3,1
7,2021,2021-09-12,Lions,49ers,1,2
8,2021,2021-09-12,Texans,Jaguars,0,3
9,2021,2021-09-12,Commanders,Chargers,1,2


In [873]:
scores = scores.merge(
    turnovers,
    on=["Season", "Date", "HomeTeam", "AwayTeam"],
    how="left"
)

scores.head(15)

,Season,Week,GameStatus,Day,Date,AwayTeam,AwayRecord,AwayScore,AwayWin,HomeTeam,...,AwaySeeding,HomeSeeding,PostSeason,game_id,HomeRushYards,HomePassYards,AwayRushYards,AwayPassYards,HomeTeamTO,AwayTeamTO
0,2021,Week 1,FINAL,THU,2021-09-09,Cowboys,0-1,29.0,0.0,Buccaneers,...,NaN,NaN,0,0,51.0,368.0,60.0,393.0,4.0,1.0
1,2021,Week 1,FINAL,SUN,2021-09-12,Jaguars,0-1,21.0,0.0,Texans,...,NaN,NaN,0,1,171.0,291.0,66.0,347.0,0.0,3.0
2,2021,Week 1,FINAL,SUN,2021-09-12,Chargers,1-0,20.0,1.0,Commanders,...,NaN,NaN,0,2,127.0,135.0,90.0,337.0,1.0,2.0
3,2021,Week 1,FINAL,SUN,2021-09-12,Seahawks,1-0,28.0,1.0,Colts,...,NaN,NaN,0,3,113.0,251.0,139.0,254.0,1.0,1.0
4,2021,Week 1,FINAL,SUN,2021-09-12,Jets,0-1,14.0,0.0,Panthers,...,NaN,NaN,0,4,111.0,279.0,46.0,258.0,1.0,1.0
5,2021,Week 1,FINAL,SUN,2021-09-12,Vikings,0-1,24.0,0.0,Bengals,...,NaN,NaN,0,5,149.0,261.0,53.0,362.0,0.0,1.0
6,2021,Week 1,FINAL,SUN,2021-09-12,Cardinals,1-0,38.0,1.0,Titans,...,NaN,NaN,0,6,71.0,212.0,116.0,297.0,3.0,1.0
7,2021,Week 1,FINAL,SUN,2021-09-12,49ers,1-0,41.0,1.0,Lions,...,NaN,NaN,0,7,117.0,377.0,131.0,338.0,1.0,2.0
8,2021,Week 1,FINAL,SUN,2021-09-12,Steelers,1-0,23.0,1.0,Bills,...,NaN,NaN,0,8,133.0,270.0,75.0,188.0,1.0,0.0
9,2021,Week 1,FINAL,SUN,2021-09-12,Eagles,1-0,32.0,1.0,Falcons,...,NaN,NaN,0,9,125.0,164.0,172.0,279.0,0.0,0.0


In [874]:
#take out pro bowl and cancelled games
scores = scores[(~scores['Week'].str.contains(r"pro bowl", case=False, na=False)) & (~scores['GameStatus'].str.contains('CANCELLED'))]

scores[scores["HomeTeamTO"].isna()]

,Season,Week,GameStatus,Day,Date,AwayTeam,AwayRecord,AwayScore,AwayWin,HomeTeam,...,AwaySeeding,HomeSeeding,PostSeason,game_id,HomeRushYards,HomePassYards,AwayRushYards,AwayPassYards,HomeTeamTO,AwayTeamTO


In [875]:
#Check for missing turnover data --> shows if merge was successful
scores[["HomeTeamTO", "AwayTeamTO"]].isna().mean()

HomeTeamTO    0.0
AwayTeamTO    0.0
dtype: float64

In [876]:
scores[scores['AwayPassYards'].isna()]

,Season,Week,GameStatus,Day,Date,AwayTeam,AwayRecord,AwayScore,AwayWin,HomeTeam,...,AwaySeeding,HomeSeeding,PostSeason,game_id,HomeRushYards,HomePassYards,AwayRushYards,AwayPassYards,HomeTeamTO,AwayTeamTO


In [877]:
#time to create rolling windows
#we need to be able to collect data based on team and week rather than home/away so before creating windows_df we are going to create team_games

scores = scores.copy()
scores = scores.sort_values('Date').reset_index()

home_team_games = scores[['Season', 'game_id', 'Week', 'Date', 'HomeTeam', 'AwayTeam', 'HomeRushYards', 'AwayRushYards', 'HomePassYards', 'AwayPassYards', 'HomeTeamTO', 'AwayTeamTO', 'HomeScore', 'AwayScore', 'HomeRecord', 'AwayRecord', 'HomeWin']]
home_team_games = home_team_games.rename(columns={
    'HomeTeam': 'Team',
    'AwayTeam': 'Opponent',
    'HomeRushYards': 'RushYards',
    'AwayRushYards': 'RushYardsAllowed',
    'HomePassYards': 'PassYards',
    'AwayPassYards': 'PassYardsAllowed',
    'HomeTeamTO': 'Giveaways',
    'AwayTeamTO': 'Takeaways',
    'HomeScore': 'PointsScored',
    'AwayScore': 'PointsAllowed',
    'HomeRecord': 'TeamRecord',
    'HomeWin': 'Win'})

away_team_games = scores[['Season', 'game_id', 'Week', 'Date', 'AwayTeam', 'HomeTeam', 'AwayRushYards', 'HomeRushYards', 'AwayPassYards', 'HomePassYards',  'AwayTeamTO', 'HomeTeamTO', 'AwayScore', 'HomeScore', 'AwayRecord', 'HomeRecord', 'AwayWin']]
away_team_games = away_team_games.rename(columns={
    'AwayTeam': 'Team',
    'HomeTeam': 'Opponent',
    'AwayRushYards': 'RushYards',
    'HomeRushYards': 'RushYardsAllowed',
    'AwayPassYards': 'PassYards',
    'HomePassYards': 'PassYardsAllowed',
    'AwayTeamTO': 'Giveaways',
    'HomeTeamTO': 'Takeaways',
    'AwayScore': 'PointsScored',
    'HomeScore': 'PointsAllowed',
    'AwayRecord': 'TeamRecord',
    'HomeRecord': 'OpponentRecord',
    'AwayWin': 'Win'})

home_team_games['HomeFieldAdvantage'] = 1
away_team_games['HomeFieldAdvantage'] = 0

team_games = pd.concat([home_team_games, away_team_games], ignore_index=True)
team_games.sort_values(['Team', 'Date'], inplace=True)
data_2021_2024 = team_games.copy()
data_2021_2024.sort_values(['Season', 'Week', 'Date'], inplace=True)
data_2021_2024.to_csv('../data/processed/nfl_cleaned_stats(2021-2024).csv', index=False)

In [878]:
#Create rolling windows dataframe
windows = [3, 5, 7]
stats = [
    "RushYards", "RushYardsAllowed",
    "PassYards", "PassYardsAllowed",
    "Giveaways", "Takeaways",
    "PointsScored", "PointsAllowed", "Win"]

for w in windows:
    for s in stats:
        team_games[f"{s}{w}GameAvg"] = team_games.groupby(['Season', 'Team']).shift(1).rolling(window=w, min_periods=w)[s].mean()

rolling_cols = [col for col in team_games.columns if 'Avg' in col]
id_cols = ['Season', 'game_id', 'Team', 'Opponent', 'Date', 'Week', 'HomeFieldAdvantage']
target_cols = ['PointsScored', 'PointsAllowed', 'Win']
team_games.head(8)

,Season,game_id,Week,Date,Team,Opponent,RushYards,RushYardsAllowed,PassYards,PassYardsAllowed,...,Win5GameAvg,RushYards7GameAvg,RushYardsAllowed7GameAvg,PassYards7GameAvg,PassYardsAllowed7GameAvg,Giveaways7GameAvg,Takeaways7GameAvg,PointsScored7GameAvg,PointsAllowed7GameAvg,Win7GameAvg
1149,2021,7,Week 1,2021-09-12,49ers,Lions,131.0,117.0,338.0,377.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1162,2021,20,Week 2,2021-09-19,49ers,Eagles,148.0,166.0,189.0,190.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,2021,46,Week 3,2021-09-26,49ers,Packers,67.0,100.0,272.0,261.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,2021,58,Week 4,2021-10-03,49ers,Seahawks,142.0,105.0,312.0,149.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1203,2021,76,Week 5,2021-10-10,49ers,Cardinals,156.0,97.0,207.0,237.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,2021,105,Week 7,2021-10-24,49ers,Colts,111.0,146.0,171.0,150.0,...,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1250,2021,113,Week 8,2021-10-31,49ers,Bears,145.0,161.0,312.0,175.0,...,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,2021,133,Week 9,2021-11-07,49ers,Cardinals,39.0,160.0,301.0,278.0,...,0.2,128.571429,127.428571,257.285714,219.857143,1.571429,0.714286,24.0,24.428571,0.428571


In [879]:
# 1. Pick a specific team (e.g., 'KC') and sort by date
debug_df = team_games[team_games['Team'] == '49ers'].sort_values(['Season', 'Week'])

# 2. Look at the raw stat and your rolling feature side-by-side
# Replace 'Points' and 'Rolling_Points' with your actual column names
cols_to_check = ['Week', 'PassYards', 'PassYards3GameAvg']
#print(debug_df[cols_to_check].head(10))
team_games[team_games['Team'] == '49ers'][cols_to_check].head(10)

,Week,PassYards,PassYards3GameAvg
1149,Week 1,338.0,NaN
1162,Week 2,189.0,NaN
34,Week 3,272.0,NaN
53,Week 4,312.0,266.333333
1203,Week 5,207.0,257.666667
104,Week 7,171.0,263.666667
1250,Week 8,312.0,230.000000
125,Week 9,301.0,230.000000
149,Week 10,182.0,261.333333
1298,Week 11,176.0,265.000000


In [880]:
def nan_check(df):
    for col in df.columns:
        if df[col].isna().sum() > 0:
            print(f"Column {col} has {df[col].isna().sum()} missing values.")

nan_check(scores)

Column AwaySeeding has 1087 missing values.
Column HomeSeeding has 1087 missing values.


In [881]:
scores[scores['HomeRushYards'].isna()]

,index,Season,Week,GameStatus,Day,Date,AwayTeam,AwayRecord,AwayScore,AwayWin,...,AwaySeeding,HomeSeeding,PostSeason,game_id,HomeRushYards,HomePassYards,AwayRushYards,AwayPassYards,HomeTeamTO,AwayTeamTO


In [882]:
team_games = team_games.sort_values(['Season', 'Date']).copy()
team_games['game_num'] = team_games.groupby(['Season', 'Team']).cumcount() + 1
team_games.head(10)

,Season,game_id,Week,Date,Team,Opponent,RushYards,RushYardsAllowed,PassYards,PassYardsAllowed,...,RushYards7GameAvg,RushYardsAllowed7GameAvg,PassYards7GameAvg,PassYardsAllowed7GameAvg,Giveaways7GameAvg,Takeaways7GameAvg,PointsScored7GameAvg,PointsAllowed7GameAvg,Win7GameAvg,game_num
0,2021,0,Week 1,2021-09-09,Buccaneers,Cowboys,51.0,60.0,368.0,393.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1135,2021,0,Week 1,2021-09-09,Cowboys,Buccaneers,60.0,51.0,393.0,368.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1149,2021,7,Week 1,2021-09-12,49ers,Lions,131.0,117.0,338.0,377.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1136,2021,14,Week 1,2021-09-12,Bears,Rams,134.0,89.0,231.0,321.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,2021,5,Week 1,2021-09-12,Bengals,Vikings,149.0,53.0,261.0,362.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,2021,8,Week 1,2021-09-12,Bills,Steelers,133.0,75.0,270.0,188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1138,2021,12,Week 1,2021-09-12,Broncos,Giants,170.0,68.0,286.0,267.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1139,2021,10,Week 1,2021-09-12,Browns,Chiefs,163.0,73.0,321.0,337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1143,2021,6,Week 1,2021-09-12,Cardinals,Titans,116.0,71.0,297.0,212.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1147,2021,2,Week 1,2021-09-12,Chargers,Commanders,90.0,127.0,337.0,135.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [883]:
team_games = team_games[team_games['game_num'] > 7].copy()
team_games_rolling = team_games[id_cols + rolling_cols + target_cols].copy()

In [884]:
home_feat = team_games_rolling.merge(scores[['Season', 'game_id', 'HomeTeam']], left_on=['Season', 'game_id', 'Team'], right_on=['Season', 'game_id', 'HomeTeam'], how='inner')
home_feat = home_feat.drop('HomeFieldAdvantage', axis=1)
home_feat = home_feat[['Season', 'game_id'] + rolling_cols + target_cols]
home_feat = home_feat.rename(columns={c: f'Home{c}' for c in rolling_cols + target_cols})

away_feat = team_games_rolling.merge(scores[['Season', 'game_id', 'AwayTeam']], left_on=['Season', 'game_id', 'Team'], right_on=['Season', 'game_id', 'AwayTeam'], how='inner')
away_feat = away_feat.drop('HomeFieldAdvantage', axis=1)
away_feat = away_feat[['Season', 'game_id'] + rolling_cols]
away_feat = away_feat.rename(columns={c: f'Away{c}' for c in rolling_cols})

In [885]:
display(home_feat.head())
display(away_feat.head())
print(home_feat.shape)
print(away_feat.shape)

,Season,game_id,HomeRushYards3GameAvg,HomeRushYardsAllowed3GameAvg,HomePassYards3GameAvg,HomePassYardsAllowed3GameAvg,HomeGiveaways3GameAvg,HomeTakeaways3GameAvg,HomePointsScored3GameAvg,HomePointsAllowed3GameAvg,...,HomePassYards7GameAvg,HomePassYardsAllowed7GameAvg,HomeGiveaways7GameAvg,HomeTakeaways7GameAvg,HomePointsScored7GameAvg,HomePointsAllowed7GameAvg,HomeWin7GameAvg,HomePointsScored,HomePointsAllowed,HomeWin
0,2021,107,135.666667,90.333333,242.333333,204.000000,0.666667,1.666667,28.333333,9.666667,...,290.857143,230.142857,0.857143,2.000000,32.142857,16.285714,1.000000,21.0,24.0,0.0
1,2021,113,137.666667,141.333333,164.000000,209.000000,2.000000,0.666667,12.333333,23.666667,...,159.428571,248.428571,1.428571,1.142857,14.428571,23.142857,0.428571,22.0,33.0,0.0
2,2021,118,81.666667,137.666667,269.666667,264.333333,2.000000,0.333333,19.000000,26.000000,...,264.142857,236.571429,1.285714,0.857143,20.000000,18.142857,0.428571,17.0,10.0,1.0
3,2021,111,163.333333,97.000000,258.000000,274.000000,1.000000,0.666667,24.333333,32.666667,...,242.142857,235.285714,1.000000,0.714286,24.714286,23.571429,0.571429,10.0,15.0,0.0
4,2021,109,147.666667,105.666667,263.333333,287.000000,1.000000,2.666667,28.666667,17.333333,...,247.857143,253.714286,1.000000,2.285714,24.142857,21.285714,0.428571,31.0,34.0,0.0


,Season,game_id,AwayRushYards3GameAvg,AwayRushYardsAllowed3GameAvg,AwayPassYards3GameAvg,AwayPassYardsAllowed3GameAvg,AwayGiveaways3GameAvg,AwayTakeaways3GameAvg,AwayPointsScored3GameAvg,AwayPointsAllowed3GameAvg,...,AwayWin5GameAvg,AwayRushYards7GameAvg,AwayRushYardsAllowed7GameAvg,AwayPassYards7GameAvg,AwayPassYardsAllowed7GameAvg,AwayGiveaways7GameAvg,AwayTakeaways7GameAvg,AwayPointsScored7GameAvg,AwayPointsAllowed7GameAvg,AwayWin7GameAvg
0,2021,107,114.333333,144.666667,271.000000,241.000000,0.666667,1.666667,24.333333,15.333333,...,1.0,103.142857,122.428571,253.428571,231.857143,0.857143,1.571429,24.000000,20.857143,0.857143
1,2021,108,118.666667,94.666667,330.333333,277.000000,1.333333,0.666667,32.333333,17.666667,...,0.8,106.857143,91.142857,288.285714,263.857143,1.285714,0.857143,27.000000,18.285714,0.714286
2,2021,119,141.000000,93.333333,325.000000,191.333333,0.666667,2.666667,37.000000,14.000000,...,0.8,102.285714,65.857143,329.285714,285.571429,1.000000,2.000000,33.285714,21.000000,0.857143
3,2021,118,139.666667,82.666667,237.666667,316.666667,2.000000,2.000000,15.000000,29.333333,...,0.2,133.428571,101.714286,236.571429,311.142857,1.571429,1.142857,20.857143,30.000000,0.285714
4,2021,115,82.666667,91.666667,309.333333,370.333333,1.666667,1.000000,21.666667,32.666667,...,0.0,80.000000,118.285714,246.714286,328.142857,1.571429,1.285714,18.142857,29.571429,0.142857


(684, 32)
(690, 29)


In [886]:
model_df = home_feat.merge(away_feat, on=['Season', 'game_id'], how='inner')
model_df

,Season,game_id,HomeRushYards3GameAvg,HomeRushYardsAllowed3GameAvg,HomePassYards3GameAvg,HomePassYardsAllowed3GameAvg,HomeGiveaways3GameAvg,HomeTakeaways3GameAvg,HomePointsScored3GameAvg,HomePointsAllowed3GameAvg,...,AwayWin5GameAvg,AwayRushYards7GameAvg,AwayRushYardsAllowed7GameAvg,AwayPassYards7GameAvg,AwayPassYardsAllowed7GameAvg,AwayGiveaways7GameAvg,AwayTakeaways7GameAvg,AwayPointsScored7GameAvg,AwayPointsAllowed7GameAvg,AwayWin7GameAvg
0,2021,107,135.666667,90.333333,242.333333,204.000000,0.666667,1.666667,28.333333,9.666667,...,1.0,103.142857,122.428571,253.428571,231.857143,0.857143,1.571429,24.000000,20.857143,0.857143
1,2021,118,81.666667,137.666667,269.666667,264.333333,2.000000,0.333333,19.000000,26.000000,...,0.2,133.428571,101.714286,236.571429,311.142857,1.571429,1.142857,20.857143,30.000000,0.285714
2,2021,109,147.666667,105.666667,263.333333,287.000000,1.000000,2.666667,28.666667,17.333333,...,0.8,151.571429,99.714286,251.000000,289.571429,1.285714,1.142857,27.571429,23.428571,0.714286
3,2021,112,89.333333,103.333333,229.333333,300.666667,1.666667,1.000000,15.666667,27.000000,...,0.2,118.285714,135.428571,254.142857,239.857143,1.000000,1.000000,22.714286,26.428571,0.285714
4,2021,110,69.000000,154.333333,229.333333,241.666667,1.666667,1.000000,10.000000,29.000000,...,0.8,94.714286,110.142857,310.285714,285.000000,1.000000,1.714286,29.571429,20.857143,0.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,2024,1132,157.000000,81.333333,276.333333,307.666667,0.666667,1.333333,35.000000,20.000000,...,1.0,155.285714,126.000000,234.142857,216.428571,1.571429,1.142857,28.571429,24.000000,0.857143
669,2024,1134,143.000000,86.666667,212.666667,196.666667,0.000000,1.333333,29.000000,14.666667,...,1.0,218.285714,74.857143,216.714286,205.857143,0.285714,0.857143,30.285714,14.857143,0.857143
670,2024,1133,133.333333,107.333333,209.000000,187.000000,0.000000,3.000000,27.666667,10.000000,...,0.8,108.857143,109.000000,212.857143,242.571429,0.285714,1.000000,23.000000,17.000000,0.857143
671,2024,1136,56.333333,168.000000,198.333333,253.666667,0.000000,0.666667,17.333333,20.666667,...,0.8,154.714286,104.857143,232.285714,271.285714,0.142857,1.571429,32.571429,25.142857,0.714286


In [887]:
model_df[model_df['HomeRushYards3GameAvg'].isna()]

,Season,game_id,HomeRushYards3GameAvg,HomeRushYardsAllowed3GameAvg,HomePassYards3GameAvg,HomePassYardsAllowed3GameAvg,HomeGiveaways3GameAvg,HomeTakeaways3GameAvg,HomePointsScored3GameAvg,HomePointsAllowed3GameAvg,...,AwayWin5GameAvg,AwayRushYards7GameAvg,AwayRushYardsAllowed7GameAvg,AwayPassYards7GameAvg,AwayPassYardsAllowed7GameAvg,AwayGiveaways7GameAvg,AwayTakeaways7GameAvg,AwayPointsScored7GameAvg,AwayPointsAllowed7GameAvg,AwayWin7GameAvg


In [888]:
linear_model_df = model_df.drop(columns=['Season', 'game_id', 'HomeWin'])
linear_design_matrix = linear_model_df.drop(columns=['HomePointsScored','HomePointsAllowed'])
linear_target = linear_model_df['HomePointsScored'] - linear_model_df['HomePointsAllowed']

In [889]:
home_cols = [c for c in linear_design_matrix.columns if c.startswith("Home")]

diff_df = pd.DataFrame(index=linear_design_matrix.index)
for home_col in home_cols:
    away_col = home_col.replace("Home", "Away")
    if away_col in linear_design_matrix.columns:
        diff_name = home_col.replace("Home", "Diff")
        diff_df[diff_name] = linear_design_matrix[home_col] - linear_design_matrix[away_col]

linear_design_matrix = diff_df
modeling_df = linear_design_matrix.copy()
modeling_df['Target'] = linear_target
modeling_df['Season'] = model_df['Season']

In [890]:
modeling_df.to_csv('../data/processed/nfl_model_features&target_v1.csv', index=False)

In [891]:
nan_check(linear_design_matrix)

In [892]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [893]:
# X = modeling_df.drop(['Target', 'Season'], axis=1).copy()
# y = modeling_df['Target'].copy()

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# model = LinearRegression()
# model.fit(X_train_scaled, y_train)

# y_pred_train = model.predict(X_train_scaled)
# y_pred = model.predict(X_test_scaled)
# mae = mean_absolute_error(y_test, y_pred)
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f'Test Error (MAE): {mae}')
# print(f'Test Error (RMSE): {rmse}')

# train_mae = mean_absolute_error(y_train, y_pred_train)
# train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
# print(f'Training Error (MAE): {train_mae}')
# print(f'Training Error (RMSE): {train_rmse}')

Test Error (MAE): 10.574581432404345
Test Error (RMSE): 13.260579652339448
Training Error (MAE): 9.79137163018518
Training Error (RMSE): 12.796464699692267
